In [ ]:
# import requests
# from azure.identity import ClientSecretCredential

# # 🔹 Azure AD App Credentials (Replace with your values)
TENANT_ID = ""
CLIENT_ID = ""
CLIENT_SECRET = ""

# # 🔹 Microsoft Graph API Scope
# GRAPH_SCOPE = "https://graph.microsoft.com/.default"

# # 🔹 Step 1: Authenticate & Get Access Token
# def get_access_token():
#     credential = ClientSecretCredential(
#         tenant_id=TENANT_ID,
#         client_id=CLIENT_ID,
#         client_secret=CLIENT_SECRET
#     )
#     token = credential.get_token(GRAPH_SCOPE).token
#     return token

# # 🔹 Step 2: Fetch Available SKUs
# def get_skus(access_token):
#     url = "https://graph.microsoft.com/v1.0/subscribedSkus"
#     headers = {"Authorization": f"Bearer {access_token}"}
    
#     response = requests.get(url, headers=headers)
#     response.raise_for_status()
    
#     data = response.json()
    
#     print("🔹 Available SKUs in Your Organization:")
#     for sku in data["value"]:
#         print(f"SKU ID: {sku['skuId']} | Product Name: {sku['skuPartNumber']}")

# # Run the script
# if __name__ == "__main__":
#     token = get_access_token()
#     get_skus(token)


StatementMeta(, b5c0144e-0fb1-428d-a0bb-9d5dff68f8e8, 6, Finished, Available, Finished)

🔹 Available SKUs in Your Organization:
SKU ID: c5928f49-12ba-48f7-ada3-0d743a3601d5 | Product Name: VISIOCLIENT
SKU ID: 1f2f344a-700d-42c9-9427-5cea1d5d7ba6 | Product Name: STREAM
SKU ID: 2b317a4a-77a6-4188-9437-b68a77b4e2c6 | Product Name: INTUNE_A_D
SKU ID: 488ba24a-39a9-4473-8ee5-19291e71b002 | Product Name: WIN10_VDA_E5
SKU ID: b05e124f-c7cc-45a0-a6aa-8cf78c946968 | Product Name: EMSPREMIUM
SKU ID: 1e615a51-59db-4807-9957-aa83c3657351 | Product Name: Dynamics_365_Customer_Service_Enterprise_viral_trial
SKU ID: 3dd6cf57-d688-4eed-ba52-9e40b5468c3e | Product Name: THREAT_INTELLIGENCE
SKU ID: 6ec92958-3cc1-49db-95bd-bc6b3798df71 | Product Name: Dynamics_365_Sales_Premium_Viral_Trial
SKU ID: 4b74a65c-8b4a-4fc8-9f6b-5177ed11ddfa | Product Name: VIRTUAL_AGENT_USL
SKU ID: 7a551360-26c4-4f61-84e6-ef715673e083 | Product Name: MICROSOFT_REMOTE_ASSIST
SKU ID: 4a51bf65-409c-4a91-b845-1121b571cc9d | Product Name: FLOW_PER_USER
SKU ID: 29fcd665-d8d1-4f34-8eed-3811e3fca7b3 | Product Name: Dynamic

In [ ]:
import requests
import pandas as pd
from azure.identity import ClientSecretCredential

# 🔹 Azure AD App Credentials (Replace with your values)
TENANT_ID = ""
CLIENT_ID = ""
CLIENT_SECRET = ""

# 🔹 Microsoft Graph API Scope & URL
GRAPH_SCOPE = "https://graph.microsoft.com/.default"
USERS_API_URL = "https://graph.microsoft.com/v1.0/users?$select=id,displayName,userPrincipalName,assignedLicenses"

# 🔹 Power BI Pro SKU ID
POWER_BI_PRO_SKU_ID = "f8a1db68-be16-40ed-86d5-cb42ce701560"

# 🔹 Step 1: Authenticate & Get Access Token
def get_access_token():
    credential = ClientSecretCredential(
        tenant_id=TENANT_ID,
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET
    )
    token = credential.get_token(GRAPH_SCOPE).token
    return token

# 🔹 Step 2: Get Users & Filter Those with Power BI Pro
def get_pro_users(access_token):
    headers = {"Authorization": f"Bearer {access_token}"}
    url = USERS_API_URL
    pro_users = []

    while url:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()

        for user in data.get("value", []):
            assigned_skus = [sku["skuId"] for sku in user.get("assignedLicenses", [])]
            if POWER_BI_PRO_SKU_ID in assigned_skus:  # 🔹 Check if Pro license is assigned Add Or condiation if there are multiple SKU's
                pro_users.append({
                    "User ID": user["id"],
                    "Name": user["displayName"],
                    "Email": user["userPrincipalName"]
                })

        url = data.get("@odata.nextLink")  # 🔹 Handle pagination

    return pro_users
test=pd.DataFrame()
# 🔹 Step 3: Fetch Data & Save to CSV

access_token = get_access_token()
pro_users = get_pro_users(access_token)
if pro_users:
    df_prousers = pd.DataFrame(pro_users)
    # df.to_csv("PowerBI_Pro_Users.csv", index=False)
    print("✅ Power BI Pro user list saved to 'PowerBI_Pro_Users.csv'")
    # display(df)
        
else:
    print("⚠️ No users found with Power BI Pro license.")




StatementMeta(, b5c0144e-0fb1-428d-a0bb-9d5dff68f8e8, 11, Finished, Available, Finished)

✅ Power BI Pro user list saved to 'PowerBI_Pro_Users.csv'


In [10]:
from pyspark.sql import SparkSession

path = "abfss://d3120490-76ae-4ef4-a440-2bd65732ccdc@onelake.dfs.fabric.microsoft.com/fecab367-5d3a-41c1-8037-7801192932ba/Files"
# Initialize Spark Session (If Not Already Started)
spark = SparkSession.builder.appName("DeltaTableWrite").getOrCreate()

df_prousers.to_parquet(path+"/prousers.parquet")

# Read Parquet File from Fabric OneLake
df_new = spark.read.parquet(
    path+"prousers.parquet"
)

# Enable Delta Column Mapping by Name
spark.conf.set("spark.databricks.delta.properties.defaults.columnMapping.mode", "name")

# Write Data as Delta Table with Overwrite Mode
df_new.write.mode("overwrite").format("delta").option("delta.columnMapping.mode", "name").saveAsTable("PowerBIProUsers")

print("✅ Delta table 'PowerBIProUsers' successfully created with column mapping mode 'name'.")


StatementMeta(, b5c0144e-0fb1-428d-a0bb-9d5dff68f8e8, 12, Finished, Available, Finished)

✅ Delta table 'PowerBIProUsers' successfully created with column mapping mode 'name'.
